In [0]:
from __future__ import print_function

from models.ResNet_3D.model import generate_model
from models.ResNet_3D.opts import create_args
from models.ResNet_3D.mean import get_mean, get_std

import torch

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scripts.read_video import generate_block
import time
import os
import argparse

In [0]:
def find_length(filename, seg_length=1):
    """
    Function to find the length of the video in terms of specified segments.
    """
    return sum(1 for _ in generate_block(filename, 1))

In [0]:
def generate_features(resnet, filename):
    """
    Function to process the current video file and generate the c3d fetures.
    
    Parameters
    resnet          :torch.nn.Module
                     The c3d feature extraction model.
    filename        :str
                     The filename of the videos to return the features
    
    Returns
    -------
    features        :numpy.array
                     The features of the video in the required shape.
    """
    total_length = find_length(filename)
    block = generate_block(filename, 1)
    start_time = time.time()
    feature_arr = []
    for i, curr_block in enumerate(block):
        print("{}/{}".format(i+1, total_length), sep='\r', end='\r')
        features = resnet(curr_block['block'])
        features = (features - features.mean())/(features.max() - features.mean())
        feature_arr.append(features.cpu().detach().numpy())
        del features
    print("")
    total_time = time.time()-start_time
    features = np.rollaxis(np.array(feature_arr), 1 )[0]
    return features

In [0]:
def iterate_folder(base_path, c3d):
    """
    Function to iterate over the video files in the given folder
    and call the function to generate the c3d features and write the
    features in a csv file.
    Parameters
    ----------
    base_path       :str
                     Path of the folder containing the video files.
    resnet          :torch.nn.Module
                     The resnet feature extraction model.
    Returns
    -------
    None
    """
    start_time = time.time()
    total_files = len(os.listdir(base_path))
    category = base_path.split('/')[-1]
    print("Calculating features for :", category)

    for i, vid_file in enumerate(os.listdir(base_path)):
        features = generate_features(c3d, os.path.join(base_path, vid_file))
        mult = len(features)//32
        feature_norm_arr = []
        for i in range(32):
            sub_arr = np.linalg.norm(features[i*mult:(i+1)*mult], axis=0)
            feature_norm_arr.append(sub_arr)
        folder_path = os.path.join("results", "features", category)
        file_path = os.path.join(folder_path, vid_file.split('.')[0]+'.csv')
        if not(os.path.exists(folder_path)):
            os.mkdir(folder_path)
        pd.DataFrame(np.array(feature_norm_arr, dtype='float16'), index=None).to_csv(file_path, header=False)
                     
        print("{}/{} completed".format(i+1, total_files), sep='\r', end='\r')

In [0]:
opt = create_args(model_name = 'resnext', model_depth=101, \
                  model='/content/drive/My Drive/PretrainedModels/resnext-101-kinetics.pth')
opt.mean = get_mean()
opt.std = get_std()
opt.arch = '{}-{}'.format(opt.model_name, opt.model_depth)
opt.sample_size = 112
opt.sample_duration = 16
opt.n_classes = 400

In [21]:
model = generate_model(opt)
print('loading model {}'.format(opt.model))
model_data = torch.load(opt.model)
assert opt.arch == model_data['arch']
model.load_state_dict(model_data['state_dict'],strict=False)
model = model.eval()

loading model /content/drive/My Drive/PretrainedModels/resnext-101-kinetics.pth


In [45]:
#parser = argparse.ArgumentParser()
#parser.add_argument("--base_path", type=str, default='../SampleVideos/videos')
#parser.add_argument("--c3d_weights", type=str, default='../weights/c3d.pickle')
#args = parser.parse_args()
base_path = 'SampleVideos/videos/'#args.base_path
iterate_folder(base_path, model)

Calculating features for : 
42
42/42
Time taken =  6.183789968490601
